In [1]:
from pyMSOO.MFEA.model import MaTGA
from pyMSOO.utils.Crossover import *
from pyMSOO.utils.Mutation import *
from pyMSOO.utils.Selection import *
from pyMSOO.utils.DimensionAwareStrategy import *
from pyMSOO.MFEA.benchmark.continous import *
from pyMSOO.utils.MultiRun.RunMultiTime import * 

#### Benchmark

##### CEC17

In [2]:
# tasks, IndClass = CEC17_benchmark.get_2tasks_benchmark(1)
# tasks, IndClass = WCCI22_benchmark.get_complex_benchmark(10)
tasks, IndClass = CEC17_benchmark.get_10tasks_benchmark()
# tasks, IndClass = WCCI22_benchmark.get_50tasks_benchmark(10)

In [6]:
from pyMSOO.utils.numba_utils import *
from pyMSOO.utils import Crossover, Mutation, DimensionAwareStrategy, Selection
import random
import copy

class model(AbstractModel.model):
    '''
    An Adaptive Archive-Based Evolutionary Framework for Many-Task Optimization.
    '''
    def compile(self, 
                IndClass: Type[Individual], 
                tasks: List[AbstractTask], 
                crossover: Crossover.AbstractCrossover, 
                mutation: Mutation.AbstractMutation, 
                selection: Selection.AbstractSelection, 
                dimension_strategy: DimensionAwareStrategy.AbstractDaS = DimensionAwareStrategy.NoDaS(),
                *args, 
                **kwargs):
        super().compile(IndClass, tasks, crossover, mutation, dimension_strategy, selection, *args, **kwargs)
        
    def fit(self, 
            nb_generations, 
            rmp = 0.1, 
            nb_inds_each_task = 100, 
            ro = 0.8,
            shrink_rate = 0.8,
            max_archive_size = 300,
            replace_rate = 0.2,
            evaluate_initial_skillFactor = True, 
            *args, 
            **kwargs) -> List[Individual]:
        '''
        Arguments include:\n
        + `nb_generations`: number of generations
        + `rmp`: random mating probability
        + `nb_inds_tasks`: number of individual per task; nb_inds_tasks[i] = num individual of task i
        + `ro`: 
        + `shrink_rate`: 
        + `max_archive_size`: maximum size of each archive population
        + `replace_rate`: 
        + `evaluate_initial_skillFactor`:
            + if True: individuals are initialized with skill factor most fitness with them
            + else: randomize skill factors for individuals
        '''

        super().fit(*args, **kwargs)
        
        assert max_archive_size >= nb_inds_each_task

        self.max_archive_size = max_archive_size
        self.nb_inds_each_task = nb_inds_each_task
        self.replace_rate = replace_rate
        self.ro = ro
        
        # Initial population
        self.population = Population(
            self.IndClass,
            nb_inds_tasks = [nb_inds_each_task] * len(self.tasks), 
            dim = self.dim_uss,
            list_tasks = self.tasks,
            evaluate_initial_skillFactor = evaluate_initial_skillFactor
        )

        # Initial archive population
        self.archive_size = [nb_inds_each_task] * len(self.tasks)
        self.archive_population: List[SubPopulation] = [SubPopulation(self.IndClass, skill_factor=i, dim=self.dim_uss, num_inds=nb_inds_each_task, task=task) \
                                                        for i, task in enumerate(self.tasks)]

        # Update initial archives
        self.update_archives(init=True)
        
        # save history
        self.history_cost.append([ind.fcost for ind in self.population.get_solves()])
        
        self.render_process(0, ['Cost'], [self.history_cost[-1]], use_sys = True)
        
        # Reward
        self.R = np.ones((len(self.tasks), len(self.tasks)))

        # Roulette wheel probability
        self.probability = np.zeros((len(self.tasks), len(self.tasks)))

        for epoch in range(nb_generations):
            
            for t in range(len(self.tasks)):
                
                if (random.random() > rmp): 
                    # Intra-task transfer

                    # initial offspring_population of generation
                    offsprings = Population(
                        self.IndClass,
                        nb_inds_tasks = [0] * len(self.tasks), 
                        dim = self.dim_uss,
                        list_tasks= self.tasks,
                    )

                    # update operators
                    self.crossover.update(population = self.population)
                    self.mutation.update(population = self.population)

                    for i in range(int(nb_inds_each_task/2)):
                        pa, pb = self.population[t].__getRandomItems__(2)

                        oa, ob = self.crossover(pa, pb, t, t)
                        u = np.where(np.random.random() < 0.9)[0]
                        oa.genes[u] = pa.genes[u]
                        ob.genes[u] = pb.genes[u] 

                        # mutate
                        oa = self.mutation(oa, return_newInd = True)
                        oa.skill_factor = t

                        ob = self.mutation(ob, return_newInd = True)    
                        ob.skill_factor = t

                        offsprings.__addIndividual__(oa)
                        offsprings.__addIndividual__(ob)

                    self.population = self.population + offsprings

                    # Update rank
                    self.population.update_rank()

                    # selection
                    self.selection(self.population, [nb_inds_each_task] * len(self.tasks))
                
                else:
                    # Inter-task transfer
                    
                    # Find the most assisting task 
                    t_j = self.adaptive_choose(t)

                    assert t_j != t, "Something wrong"

                    s = self.population[t].__getBestIndividual__.fcost
                    tmpS = s

                    for i in range(len(self.population[t])):
                        genes = self.population[t][i].genes.copy()

                        k = random.randint(0, self.dim_uss - 1)
                        r1 = numba_randomchoice(len(self.population[t_j]))
                        
                        idx = np.where(np.random.rand(self.dim_uss) < 0.9)[0]
                        
                        genes[idx] = self.population[t_j][r1].genes[idx].copy()
                        genes[k] = self.population[t_j][r1].genes[k].copy()
                        
                        off = self.dimension_strategy(self.IndClass(genes=genes, skill_factor=t), t_j, self.population[t][i])
                        fcost = self.tasks[t](off)
                        off.fcost = fcost
                        tmpS = min(tmpS, fcost)
                        
                        if (fcost < self.population[t][i].fcost):
                            # off = self.IndClass(genes = genes, skill_factor = t, fcost=fitness)
                            # off.skill_factor = t
                            # off.fcost = fitness
                            self.population[t][i] = off
                            
                    if (tmpS < s):
                        self.R[t][t_j] /= shrink_rate
                    else:
                        self.R[t][t_j] *= shrink_rate
            
            # Update rank
            self.population.update_rank()

            # Update archive population
            self.update_archives()

            # Update strategy
            self.dimension_strategy.update(population = self.population)

            # save history
            self.history_cost.append([ind.fcost for ind in self.population.get_solves()])

            #print
            self.render_process((epoch+1)/nb_generations, ['Cost'], [self.history_cost[-1]], use_sys= True)
        print('\nEND!')
        
        #solve
        self.last_pop = self.population
        return self.last_pop.get_solves() 

    def update_archives(self, init = True):
        '''
        Update every archive population
        '''

        if init == True:
            for t in range(len(self.tasks)):
                self.archive_population[t] = copy.copy(self.population[t])
        else:
            for t in range(len(self.tasks)):
                self.update_archive(t)
    
    def update_archive(self, task):
        '''
        Update one archive population
        '''
        
        remain_size = self.max_archive_size - self.archive_size[task]
        u = np.random.rand(self.nb_inds_each_task)
        idx = np.where(u < self.replace_rate)[0]
        
        assert len(self.archive_population[task]) == self.archive_size[task], "Wrong shape, got {} and {}".format(len(self.archive_population[task]), self.archive_size[task])

        if len(idx) <= remain_size:
            self.archive_population[task] += copy.copy(self.population[task][idx])
            self.archive_size[task] += len(idx)
        else:
            self.archive_population[task] += copy.copy(self.population[task][idx[:remain_size]])
            r = np.random.randint(0, self.max_archive_size, size = len(idx) - remain_size).tolist()
            self.archive_population[task][r] = copy.copy(self.population[task][idx[remain_size:]])
            self.archive_size[task] = self.max_archive_size

        assert len(self.archive_population[task]) == self.archive_size[task], "Wrong shape, got {} and {}".format(len(self.archive_population[task]), self.archive_size[task])
    
    def adaptive_choose(self, task):
        """
        Adaptively choose another task
        """

        # The similarity vector
        sim_vec = np.array([self.find_sim(task, i) for i in range(len(self.tasks))])

        self.probability[task] = self.ro * self.probability[task] + self.R[task]/(1 + np.log(1 + sim_vec))

        self.probability[task][task] = 0

        return numba_randomchoice_w_prob(self.probability[task]/sum(self.probability[task]))
    
    def find_sim(self, task_a, task_b):
        """
        Find similarity score
        """

        if (task_a == task_b):
            return 0
        
        subPop_a: SubPopulation = self.archive_population[task_a]
        subPop_b: SubPopulation = self.archive_population[task_b]
        D = min(self.tasks[task_a].dim, self.tasks[task_b].dim)

        genes_a = np.array([ind.genes[:D] for ind in subPop_a])
        genes_b = np.array([ind.genes[:D] for ind in subPop_b])
        
        similarity = self.KLD(genes_a, genes_b)
        # assert not np.isnan(similarity)
        if np.isnan(similarity):
            similarity = 0
            
        return similarity

    def KLD(self, genes_a, genes_b):
        """
        Find KLD(archive_a, archive_b)
        """
        
        gene_cov_a, gene_mean_a = np.cov(genes_a, rowvar = False), np.mean(genes_a, axis = 0)
        gene_cov_b, gene_mean_b = np.cov(genes_b, rowvar = False), np.mean(genes_b, axis = 0)

        assert gene_cov_b.shape[1] == gene_cov_a.shape[0], "imcompatible shape. Got {} and {}".format(gene_cov_b.shape, gene_cov_a.shape)

        inv_gene_cov_b = numba_linalgo_pinv(gene_cov_b)
        inv_gene_cov_a = numba_linalgo_pinv(gene_cov_a)

        D = gene_cov_a.shape[0]
        try:
            det_b = max(numba_linalgo_det(gene_cov_b), 0.001)
        except:
            det_b = 1e-6
        try:
            det_a = max(numba_linalgo_det(gene_cov_a), 0.001)
        except:
            det_a = 1e-6
        
        if np.isnan(det_b):
            det_b = 1e-6

        if np.isnan(det_a):
            det_a = 1e-6

        kld_a_b = np.trace(numba_dot(inv_gene_cov_b, gene_cov_a)) \
                + numba_dot(numba_dot(np.transpose(gene_mean_b - gene_mean_a), inv_gene_cov_b), (gene_mean_b - gene_mean_a)) - D\
                + np.log(det_b/det_a)
        
        kld_b_a = np.trace(numba_dot(inv_gene_cov_a, gene_cov_b)) \
                + numba_dot(numba_dot(np.transpose(gene_mean_a - gene_mean_b), inv_gene_cov_a), (gene_mean_a - gene_mean_b)) - D\
                + np.log(det_a/det_b)

        return 0.5 * (np.abs(0.5 * kld_a_b) + np.abs(0.5 * kld_b_a))

In [9]:
baseModel = model()
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    # dimension_strategy=DaS_strategy(eta= 3)
)
solve = baseModel.fit(
    nb_generations = 1000, rmp = 0.1, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

Seed: None -- Time: 03m 7.72s  100 % [====================>]  Cost: 3.63E-06  1.21E-06  7.16E-07  5.36E-12  6.25E-04  2.80E-04  4.64E-03  1.51E-03  1.80E-04  1.75E+01  ,  
END!


### Multitime model

In [4]:
baseModel = MultiTimeModel(model= MaTGA)
baseModel.compile(
    IndClass= IndClass,
    tasks= tasks,
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    dimension_strategy=DaS_strategy(eta= 3)
)
baseModel.fit(
    nb_generations = 1000, rmp = 0.1, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

In [9]:
baseModel.run(
    nb_run= 30,
    save_path= './RESULTS/MaTGA_cec17.mso'
)

Seed: 0 -- Time: 02m 15.02s  100 % [====================>]  Cost: 4.23E-05  8.45E-10  2.02E-11  3.00E-11  4.16E-03  8.40E-06  7.77E-04  1.63E-03  1.13E-05  3.11E+01  ,  
END!
Seed: 1 -- Time: 02m 29.28s  100 % [====================>]  Cost: 5.87E-06  1.39E-09  1.93E-12  2.89E-11  4.07E-04  1.03E-05  3.64E-04  1.47E-03  1.19E-05  3.95E+01  ,  
END!
Seed: 2 -- Time: 02m 17.29s  100 % [====================>]  Cost: 8.48E-06  1.56E-10  3.18E-11  1.93E-12  4.94E-04  3.65E-06  1.03E-03  1.53E-03  8.82E-06  4.08E+01  ,  
END!
Seed: 3 -- Time: 02m 21.81s  100 % [====================>]  Cost: 5.02E-06  8.77E-10  2.49E-12  1.67E-09  3.14E-04  8.87E-06  4.62E-04  1.76E-03  1.66E-05  4.66E+01  ,  
END!
Seed: 4 -- Time: 02m 6.43s  100 % [====================>]  Cost: 4.42E-06  1.52E-09  6.30E-12  8.28E-11  3.95E-04  1.05E-05  5.77E-04  8.31E-04  9.61E-06  4.13E+01  ,  
END!
Seed: 5 -- Time: 02m 17.94s  100 % [====================>]  Cost: 9.20E-07  3.17E-09  3.10E-12  5.29E-11  5.52E-05  1.72E-05  

In [11]:
from pyMSOO.utils.LoadSaveModel.LoadModel import loadModel

baseModel = loadModel('./RESULTS/MaTGA_cec17.mso', ls_tasks= tasks, set_attribute= True)

Error when create object DictType. Using Temp class instead
Error when create object UnicodeType. Using Temp class instead
Error when create object Float. Using Temp class instead
Error when create object Tuple. Using Temp class instead
Error when create object UnionType. Using Temp class instead
Error while create instance model for <class 'pyMSOO.utils.EA.Population'>. Using Temp class instead
Error while create instance model for <class 'pyMSOO.utils.EA.SubPopulation'>. Using Temp class instead


In [ ]:
baseModel.render_history()
print()

In [3]:
import numpy as np

In [5]:
np.log(0/1e-6)

C:\Users\Acer\AppData\Local\Temp\ipykernel_11796\3566917858.py:1: RuntimeWarning: divide by zero encountered in log
  np.log(0/1e-6)


-inf

In [10]:
import pandas as pd

df = pd.read_csv(r"F:\BTVN\DSAI\Optimization Lab\Paper\Efficient knowledge transfer\history_cost_summaries_MaTGA3.csv")

In [19]:
sum(df['MaTGA_Tung'][-50:] < df["MaTGA"][-50:])

0

In [20]:
res = []
# for i in range(1, 11):
t, ic = WCCI22_benchmark.get_50tasks_benchmark(10)
baseModel = model()
baseModel.compile(
    IndClass= ic,
    tasks= t,
    # crossover = KL_SBXCrossover(nc= 2, k= 100, conf_thres= 1),
    crossover= SBX_Crossover(nc = 2),
    mutation= PolynomialMutation(nm = 5),
    selection= ElitismSelection(),
    dimension_strategy= DaS_strategy()
)
solve = baseModel.fit(
    nb_generations = 1000, nb_inds_each_task= 100, 
    bound_pop= [0, 1], evaluate_initial_skillFactor= True
)

# res.append([subpop.getSolveInd().fcost for subpop in baseModel.last_pop])

Seed: None -- Time: 29m 39.52s  100 % [====================>]  Cost: 5.63E-03  3.92E+02  6.79E-04  1.02E+00  1.20E+04  3.34E-03  3.88E+02  4.97E-03  7.22E-01  5.61E+03  1.73E-02  3.53E+02  9.83E-04  5.00E+00  1.27E+04  4.49E-03  3.45E+02  3.30E-03  1.72E+00  1.17E+04  2.61E-03  3.82E+02  1.47E-03  4.44E+00  1.26E+04  1.17E-02  3.90E+02  1.08E-03  1.07E+00  1.08E+04  6.76E-03  3.97E+02  3.60E-04  5.89E+00  3.28E+03  3.94E-02  3.67E+02  3.55E-04  7.33E+00  5.39E+03  9.86E-03  3.78E+02  2.72E-03  9.80E+00  1.24E+04  5.77E-03  4.10E+02  8.50E-04  5.52E+00  1.14E+04  ,  
END!


In [21]:
f = np.array([subpop.getSolveInd().fcost for subpop in baseModel.last_pop])

In [22]:
f = np.where(f < 1e-6, 0, f)

In [23]:
sum(f < df["MaTGA"][-50:])

28

In [26]:
sum(f < df["MaTGA_Tung"][-50:])

28

In [25]:
sum(df["MaTGA_Tung"][-50:] < df["MaTGA"][-50:])

0